#### importing packages

In [1]:
import pandas as pd
# import matplotlib.pyplot as plt

from carriageway import Carriageway
from impact_factor import impact
from load import ll_A, ll_70R, ll_70RT
from Reaction2 import find_bm, find_sf, find_ra, find_rb

# live load calculations

#### defining load (vehicles)
load pair contains pair (wheel position, wheel load)

In [2]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [3]:
span = 50

maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

#### make a dataframe that contains maxBMs, maxSFs_plus, maxSFs_minus

In [4]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

*how to read Excel file*

In [5]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])

# ###get index names###
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

#### impact factor

In [6]:
IF = [impact(i.name, span) for i in vehicles]
IF

[0.088, 0.088, 0.088]

#### possible combinations for given carriageway width

In [7]:
cwWidth = 6
carriageway = Carriageway(cwWidth)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

#### eccentricity calculation
a combination consists of multiple arrangements gives different eccentricities. maximum eccentricities for each combination calculated.

In [8]:
list_combinations = []
list_max_e = []
list_plot = []
for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    arrangements = i.arrangements()
    list_arrangements = []
    eccentricity = []
    max_e = 0
    for k in arrangements:
        if k.check_exceedance() * k.check_from_right() == 1:
            list_arrangements.append(k.veh)
            e = k.eccentricity()
            eccentricity.append(e)
            max_e = e if abs(e) > abs(max_e) else max_e
    list_max_e.append(max_e)

list_max_e

[-3.739698609263685e-16, -0.40500000000000025, -0.3500000000000001]

#### make a dataframe for combinations and their corresponding eccentricities

In [9]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                  index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

*export eccentricity as excel file (uncomment)*

In [10]:
# df1.to_excel('outputs/max_e.xlsx')

#### total reactions for given combination
total reactions = sum of (maxreactions at mid span * impact factor * no. of that vehicles)

In [11]:
totalBMs = [(df.loc['MaxBM'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
totalSFs = [(df.loc['MaxSF'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
combination_val, totalBMs, totalSFs

([[2, 0, 0], [0, 1, 0], [0, 0, 1]],
 [972.6023999999999, 957.2112, 734.1003999999999],
 [33.768063999999995, 32.228767999999995, 27.86168])

In [12]:
df

0.00      6.25      12.50      18.75      25.00      31.25  \
MaxBM  ClassA       0.0  2361.037  4068.075   5120.737   5526.150   5340.038   
       Class70RW    0.0  4826.025  8200.950  10182.475  10877.400  10163.225   
       Class70RT    0.0  3651.438  6259.675   7823.212   8342.050   7823.588   
MaxSF+ ClassA       0.0     0.000     0.000      0.000      0.000      0.000   
       Class70RW    0.0     0.000     0.000      0.000      0.000      0.000   
       Class70RT    0.0     0.000     0.000      0.000      0.000      0.000   
MaxSF- ClassA       0.0   -26.488   -70.584   -129.864   -191.864   -254.864   
       Class70RW    0.0   -44.812  -136.236   -252.156   -366.236   -482.156   
       Class70RT    0.0   -57.710  -143.010   -229.810   -316.610   -404.810   
MaxSF  ClassA       0.0    26.488    70.584    129.864    191.864    254.864   
       Class70RW    0.0    44.812   136.236    252.156    366.236    482.156   
       Class70RT    0.0    57.710   143.010    229.810    316.610    404.810   

                     37.50     43.75    50.00  
MaxBM  ClassA     4327.675  2494.638    0.000  
       Class70RW  8053.750  4611.975    0.000  
       Class70RT  6260.425  3652.563    0.000  
MaxSF+ ClassA        0.000     0.000    0.000  
       Class70RW     0.000     0.000    0.000  
       Class70RT     0.000     0.000    0.000  
MaxSF- ClassA     -316.864  -383.466 -452.162  
       Class70RW  -596.236  -712.156 -832.476  
       Class70RT  -491.610  -579.810 -666.610  
MaxSF  ClassA      316.864   383.466  452.162  
       Class70RW   596.236   712.156  832.476  
       Class70RT   491.610   579.810  666.610

In [13]:
df1['TotalBM'] = totalBMs
df1['TotalSF'] = totalSFs

In [14]:
df1

,ClassA,Class70Rw,Class70Rt,MaxEccentricity,TotalBM,TotalSF
comb1,2,0,0,-3.739699e-16,972.6024,33.768064
comb2,0,1,0,-4.050000e-01,957.2112,32.228768
comb3,0,0,1,-3.500000e-01,734.1004,27.861680


# loads on peir

In [15]:
span = 30
c_c = 1.5
reactions = []

max_sums = []
max_diffs=[]
lefts = []
rights = []
max_sums_at = []
max_diffs_at=[]
for veh in vehicles:
    dummy_load = list(veh.loadpair)
    no = 2
    nose_dist = 20
    nun=0
    if veh.name == 'Class 70RT':
        nose_dist = 90
        no = 1
    elif veh.name == 'Class 70RW':
        nose_dist = 31.52
        
    final_load = list(dummy_load)
    max_sum_at = 0
    for i in range(no-1):
        delta = dummy_load[-1][0] + nose_dist
        final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
         
    sumlr = 0
    nleft = 0
    nright = 0
    pleft=0
    pright=0 
    difflr=0
    
    head_at = 0
    for i in range(int(90/0.01)):
        left = 0
        right = 0
        for j in final_load:
            rb = find_rb(span,head_at- j[0], c_c)*j[1]
            ra = find_ra(span,head_at-j[0], c_c)*j[1]
            left+=rb
            right+=ra
        if left+right>sumlr:
            sumlr = left+right
            nleft, nright = left, right
            max_sum_at = head_at
        if abs(left-right)>difflr:
            difflr=abs(left-right)
            max_diff_at=head_at
        if left>pleft:
            pleft=left 
        if right>pright:
            pright=right 
        head_at+=0.01
    max_sums.append(sumlr)
    max_diffs.append(difflr)
    lefts.append(nleft)
    rights.append(nright)
    max_sums_at.append(max_sum_at)
    max_diffs_at.append(max_diff_at)

In [16]:
pd.DataFrame([lefts, rights, max_sums,max_diffs, max_sums_at,max_diffs_at], columns=['class A', '70RW', '70RT'], index=['RL', "RR", 'Max Sum','Max Difference', 'Max_sum_at', 'Max_difference_at'])




,class A,70RW,70RT
RL,255.603419,392.482051,392.422222
RR,230.902564,522.266667,297.635897
Max Sum,486.505983,914.748718,690.058120
Max Difference,395.827692,850.126496,663.025641
Max_sum_at,74.300000,38.980000,31.960000
Max_difference_at,29.990000,43.410000,29.990000


*check*

In [17]:
span = 30
c_c = 1.5
veh = ll_A
dummy_load = list(veh.loadpair)
no = 2
nose_dist = 20
if veh.name == 'Class 70RT':
    nose_dist = 90
    no = 1
elif veh.name == 'Class 70RW':
    nose_dist = 31.52

final_load = list(dummy_load)
for i in range(no-1):
    delta = dummy_load[-1][0] + nose_dist
    final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
    
left = []
right = []
head_at = 74.3

for j in final_load:
    rb = find_rb(span, head_at-j[0], c_c)*j[1]
    ra = find_ra(span, head_at-j[0], c_c)*j[1]
    left.append(rb)
    right.append(ra)

In [18]:
pd.DataFrame({'pos' : [head_at - i[0] for i in final_load], 'load' : [i[1] for i in final_load], 'left' : left, 'right' : right} )

,pos,load,left,right
0,74.3,27,0.000000,0.000000
1,73.2,27,0.000000,0.000000
2,70.0,114,0.000000,0.000000
3,68.8,114,0.000000,0.000000
4,64.5,68,0.000000,0.000000
5,61.5,68,0.000000,0.000000
6,58.5,68,0.000000,3.487179
7,55.5,68,0.000000,10.461538
8,35.5,27,0.000000,22.615385
9,34.4,27,0.000000,23.630769


In [19]:
print(find_rb(30,1,1.5,1.5))





TypeError: find_rb() takes from 2 to 3 positional arguments but 4 were given